In [4]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


In [6]:
!wget https://www.dropbox.com/s/bz8chd4rn24v3va/data.h5

--2019-02-05 13:46:53--  https://www.dropbox.com/s/bz8chd4rn24v3va/data.h5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/bz8chd4rn24v3va/data.h5 [following]
--2019-02-05 13:46:53--  https://www.dropbox.com/s/raw/bz8chd4rn24v3va/data.h5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc966b683839572843f2fa554a4d.dl.dropboxusercontent.com/cd/0/inline/AayfdSQMdTXXcrSIk6_CP18zJ0_7BYzc47lZDw71L9mj3FOH1HPz1-ZDvFV7hDzxGa37Lg7dROQWKOrUgjQ6DyJsZTEqah-cLI28qxFBS4iqJA/file# [following]
--2019-02-05 13:46:53--  https://uc966b683839572843f2fa554a4d.dl.dropboxusercontent.com/cd/0/inline/AayfdSQMdTXXcrSIk6_CP18zJ0_7BYzc47lZDw71L9mj3FOH1HPz1-ZDvFV7hDzxGa37Lg7dROQWKOrUgjQ6DyJsZTEqah-cLI28qxFBS4iqJA/file
Resolving uc966b683839572843f2fa55

In [0]:

import os
import numpy as np
import tensorflow as tf
import h5py
import math

def load_dataset():
    train_dataset = h5py.File('data.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_set_x_orig = np.array(train_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(train_dataset["test_set_y"][:]) # your test set labels
    
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig



def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y



In [49]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig = load_dataset()

X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = Y_train_orig
Y_test = Y_test_orig

print(np.array(X_train).shape)
print(np.array(X_test).shape)
print(np.array(Y_train).shape)
print(np.array(Y_test).shape)

(10370, 100, 100, 3)
(2615, 100, 100, 3)
(10370, 12)
(2615, 12)


In [0]:
def model(Input_shape):
  
  X_Input = Input(Input_shape)
  
  X = Conv2D(8, (3, 3), strides = (1,1), padding = "valid")(X_Input)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2))(X)
  
  X = Conv2D(16, (2, 2), strides = (1,1), padding = "valid")(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2))(X)
  
  X = Conv2D(32, (3, 3), strides = (1,1), padding = "valid")(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2))(X)
  
  X = Conv2D(64, (2, 2), strides = (1,1), padding = "valid")(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2))(X)
  
  X = Conv2D(128, (2, 2), strides = (1,1), padding = "valid")(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2))(X)
  
  X = Conv2D(256, (2, 2), strides = (1,1), padding = "valid")(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)
  
  X = Conv2D(512, (1, 1), strides = (1,1), padding = "valid")(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)
  
  X = Flatten()(X)
    
  X = Dense(512, activation='sigmoid')(X)
  X = Dense(256, activation='sigmoid')(X)
  X = Dense(128, activation='sigmoid')(X)
  X = Dense(64, activation='sigmoid')(X)
  X = Dense(12, activation='sigmoid')(X)
  
  model = Model(inputs = X_Input, outputs = X, name='FruitModel')
  
  return model

In [55]:
FruitsModel = model((100,100, 3))
FruitsModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 98, 98, 8)         224       
_________________________________________________________________
batch_normalization_15 (Batc (None, 98, 98, 8)         32        
_________________________________________________________________
activation_15 (Activation)   (None, 98, 98, 8)         0         
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 49, 49, 8)         0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 48, 48, 16)        528       
_________________________________________________________________
batch_normalization_16 (Batc (None, 48, 48, 16)        64        
__________

In [0]:
FruitsModel.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [57]:
FruitsModel.fit(x = X_train, y = Y_train, epochs = 20, batch_size = 50)

Epoch 1/20
10370/10370 [==============================] - 9s 894us/step - loss: 0.3063 - acc: 0.8843
Epoch 2/20
10370/10370 [==============================] - 7s 679us/step - loss: 0.0164 - acc: 0.9962
Epoch 3/20
10370/10370 [==============================] - 7s 681us/step - loss: 0.0147 - acc: 0.9964
Epoch 4/20
10370/10370 [==============================] - 7s 678us/step - loss: 0.0064 - acc: 0.9986
Epoch 5/20
10370/10370 [==============================] - 7s 681us/step - loss: 0.0111 - acc: 0.9974
Epoch 6/20
10370/10370 [==============================] - 7s 681us/step - loss: 0.0070 - acc: 0.9983
Epoch 7/20
10370/10370 [==============================] - 7s 680us/step - loss: 0.0146 - acc: 0.9954
Epoch 8/20
10370/10370 [==============================] - 7s 681us/step - loss: 0.0150 - acc: 0.9958
Epoch 9/20
10370/10370 [==============================] - 7s 671us/step - loss: 0.0151 - acc: 0.9964
Epoch 10/20
10370/10370 [==============================] - 7s 666us/step - loss: 0.0148 - a

In [59]:
preds = FruitsModel.evaluate(x = X_test, y = Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

2615/2615 [==============================] - 1s 367us/step

Loss = 7.664763660706138e-05
Test Accuracy = 1.0
